## Загрузка датасета на HuggingFace

Этот ноутбук дает пример того, как залить локальный датасет на ХФ. Адаптируйте его под свой датасет. Затем, выложите в гитлабе получившийся ноутбук (приложите к своему датасету), чтобы всегда был доступен код для заливки вашего датасета на ХФ. Убедитесь, что ячейки последовательно запускаются.

In [1]:
from PIL import Image
import json
import datasets
from tqdm import tqdm
import os

/home/jovyan/.mlspace/envs/mera_artem1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Подготовка данных

#### WARNING! 

Если ваш датасет является __ПРИВАТНЫМ__ (вы его загружали на OBS в папку private, у него закрытый тест, то есть ответы на тестовую часть мы не показываем никому), то оставьте `MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS` равным `True`. Иначе, поставьте `False`. Этот флаг дальше используется, чтобы стереть ответы перед загрузкой на ХФ датасета. На ХФ даже приватно не должно лежать датасетов с ответами!

In [27]:
MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS = True
WITH_WATERMARK = True

Возьмите датасет с OBS. Данный пример рассчитан на загрузку на ХФ локального датасета, а не напрямую из OBS.

Параметр `path_to_data` - это путь ДО файлов `shots.json` и `test.json`, которые вы будете дальше загружать на ХФ в виде датасета или домена датасета. 

Параметр `path_to_meta` - это путь ДО меты датасета.

Итоговые пути будут собираться из `path_to_data` / `path_to_meta` + `file_name.json`!

In [28]:
path_to_data = "./"
path_to_meta = "./"

Сплиты и мета лежат в формате JSON.

In [29]:
def load_json(path):
    with open(path) as f:
        data = json.load(f)
    return data

#### Подгрузка данных

Считайте сплиты и мету датасета (домена датасета). Это просто JSON файлики либо внутри прямо папки датасета, либо внутри папки по названию домена, который вы загружать будете.

In [30]:
shots = load_json(os.path.join(path_to_data, "shots.json"))["data"]
test = load_json(os.path.join(path_to_data, "test.json"))["data"]
meta = load_json(os.path.join(path_to_meta, "raw_dataset_meta.json"))

Грузим генератор промптов, как библиотеку.

На данный момент доступен в соответствующем [MR](https://gitlab.com/alenush93/mera_multi_external/-/tree/all_datasets_merged/prompt_generator): mera_multi_external/prompt_generator. Внутри папки сделать:

```bash
pip install -e .
```

In [31]:
# импорт либ
from prompt_generator import PromptGenerator
import yaml

# путь до конфига с промптами
path_to_config = "prompts_config.yaml"
# путь до описания блоков промптов внутри генератора промптов
path_to_blocks = "/home/jovyan/artem/mera_multi_external/prompt_generator/configs/prompt_blocks/templates.json"

# читаем файлы
with open(path_to_config) as f:
    prompt_configs = yaml.safe_load(f)

prompt_blocks = load_json(path_to_blocks) 

# инит генератора с переданными файлами
generator = PromptGenerator(prompt_blocks, prompt_configs, meta)

#### Обработка полей датасета

На ХФ вы загружаете датасет, где у КАЖДОГО сэмпла вместо числа в поле instruction стоит промпт. Число указывает, какой по индексу взять промпт из секции с промптами в мете датасета.

In [32]:
for card in shots:
    card["instruction"] = generator.generate_prompt(card)

for card in test:
    card["instruction"] = generator.generate_prompt(card)

Проверим полученные промпты.

In [33]:
set([elem["instruction"] for elem in test])

{'В датасете к задаче идёт такой промпт:\n\nИзображение содержит все условия математической задачи.\n\nИзображение: <image>\n\nПрошу решить задачу на основе вышеизложенного и кратко сформулировать ответ.\n\nПрошу вас подумать над решением и подробно описать ход мыслей.\n\nРассуждение напишите после слова РАССУЖДЕНИЕ, в нём кратко объясните, как вы пришли к итоговому ответу.\n\n{annotation}',
 'Внимание!\n\nВ датасете к задаче идёт такой промпт:\n\nИзображение содержит все условия математической задачи.\n\nИзображение: <image>\n\nРешить задачу на основе имеющихся данных и кратко указать ответ.\n\nПеред ответом продумать и написать ход своих мыслей.\n\nРассуждения о том, как был получен ответ на задачу, писать кратко и только после слова РАССУЖДЕНИЯ.\n\n{annotation}\n\nНа задачу не более 10 минут.\n\nПриступить!',
 'Изображение: <image>',
 'Очень бы хотелось получить решение такой задачи. Задача на знание и понимание математики школьного уровня.\n\nИмеется 1 изображение\n\nЖелательно, чт

Теперь вам нужно "обработать" вашу модальность(-и). Если у вас в датасете картинки, то для каждой вы вместо пути к картинке подгружаете саму картинку и убираете у нее `filename`.

Зачем убирать `filename`? Если этого не сделать, то потом при конвертации в байты модуль datasets увидит, что картинка взята по какому-то пути и не будет ее конвертировать в байты. А нам нужно превратить PIL.Image в байткод и загрузить на ХФ именно его.

Для картинок вы можете воспользоваться функцией `convert_images`. Она принимает на вход: 
- dataset_split: список словарей, это список сэмплов сплита (json файлик, который вы загрузили ранее)
- feature_name: строка, название поля, которое преобразуется (например, "image", "image_1")
- path_to_samples: строка. Если ваши коллеги для картинок указывали не полный путь до них, а только название файлика, то, чтобы открыть файлик, вам придется указать, как попасть в папку samples, где и находятся картинки. Тогда, укажите в path_to_samples путь от папки, где у вас этот ноутбук до папки samples (включая ее в путь)

В дополнение к подгрузке картинок нужно также наложить на них вотермарки. Делается это отдельным скриптом `watermark_simple.py`. На картинку накладывается другая картинка (лого МЕРЫ) с определенной степенью прозрачности. Скрипт допускает, как inplace замену (подгружаем картинку, накладываем вотермарку, сохраняем в датасете), так и предварительную обработку (сначала накладываем вотермарки на все картинки, сохраняем их в отдельной папке, затем просто грузим уже готовые картинки вместо изначальных).

Как запустить накладывание вотермарок для папки?
```bash
python watermark_simple.py path/to/samples/ path/to/save/samples_with_watermarks/ --watermark_path path/to/logo/mera-logo-v2.png
```

In [34]:
def convert_images(
    dataset_split: list[dict], 
    feature_name: str, 
    path_to_samples: str = None, 
    watermarked_path: str = None
):
    for card in tqdm(dataset_split):
        if path_to_samples is None:
            path_to_image = card["inputs"][feature_name]
        else:
            path_to_image = os.path.join(path_to_samples, card["inputs"][feature_name])
        
        # если заранее наложили вотермарки и сохранили картинки в другой папке
        if watermarked_path is not None:
            base, image = os.path.split(path_to_image)
            path_to_image = os.path.join(watermarked_path, image)
        
        # накладываем вотермарку прямо на ходу
        # card["inputs"][feature_name] = add_watermark_to_image(
        #     image_path=path_to_image, 
        #     watermark_processor=watermark_processor
        # )
        card["inputs"][feature_name] = Image.open(path_to_image)
        card["inputs"][feature_name].filename = ""

In [35]:
if WITH_WATERMARK:
    convert_images(shots, "image", watermarked_path="/home/jovyan/artem/mera_bucket/private/ruMathVQA/sample_watermark")
    convert_images(test, "image", watermarked_path="/home/jovyan/artem/mera_bucket/private/ruMathVQA/sample_watermark")
else:
    convert_images(shots, "image")
    convert_images(test, "image")

100%|██████████| 502/502 [00:00<00:00, 644.96it/s]


Если `exclude_fields` не пустой, то нужно удалить поля, которые там указаны.

In [36]:
meta["exclude_fields"]

{}

In [37]:
def remove_fields(dct, fields):
    res = {}
    for key in dct:
        if key in fields:
            if isinstance(fields[key], list):
                for sub_key in dct[key]:
                    if sub_key not in fields[key]:
                        res.setdefault(key, {})[sub_key] = dct[key][sub_key]
            else:
                res[key] = remove_fields(dct[key], fields[key])
        else:
            res[key] = dct[key]
    return res

In [38]:
if meta["exclude_fields"]:
    for idx, sample in enumerate(shots):
        shots[idx] = remove_fields(sample, meta["exclude_fields"])
    
    for idx, sample in enumerate(test):
        test[idx] = remove_fields(sample, meta["exclude_fields"])

#### Убираем ответы для приватных задач

Надеемся, вы поставили в начале ноутбука корректное значение `MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS`.

Если там стоит `True`, то в `test` сплите ответы на все задания стираются. Вместо них остается пустая строка, чтобы вы случайно не пушнули на ХФ датасет с заполненными ответами, и они не утекли.

In [39]:
def hide_answers(dataset_split: list[dict]):
    for card in tqdm(dataset_split):
        card["outputs"] = ""

In [40]:
if MY_DATASET_IS_PRIVATE_LETS_HIDE_ANSWERS:
    hide_answers(test)

100%|██████████| 502/502 [00:00<00:00, 1973327.66it/s]


### Создаем датасет для загрузки на ХФ

#### Аннотация полей датасета

В `features` повторяется структура КАЖДОГО сэмпла вашего датасета с описанием формата данных в каждом поле. 
- instruction всегда строка
- meta - id всегда целое число

Дальше смотрите по тому, какие поля у вашего датасета.

`features` нужен для того, чтобы ХФ сам автоматически создал техническую часть README.md датасета, заполнив ее информацией, которая используется при загрузке датасета. Отсутствие `features` может и обычно приводит к невозможности использовать датасет. Ровно такие же последствия будут от ошибок в заполнении (например, неправильно указан тип данных).

__Внимание!__ Если у вас в датасете в разных вопросах разное количество ответов, то поля в `features` нужно заполнить для сэмпла с НАИБОЛЬШИМ количеством ответов. Иначе говоря, представьте, что у вас у всех вопросов в датасете максимальное количество вариантов ответа, просто некоторые пустые. Вот из такого соображения и заполняйте `features`. Он один на весь датасет и должен охватывать все поля, которые в нем встречаются!

In [41]:
test[0]

{'instruction': 'Изображение: <image>',
 'inputs': {'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=912x848>,
  'annotation': 'Запишите ответ в формате “x:y”, x, y- целые числа.'},
 'outputs': '',
 'meta': {'id': 2,
  'categories': {'topic': 'mathematics 5th grade'},
  'image': {'source': 'photo', 'type': 'visual', 'content': 'riddle'}}}

In [42]:
features = datasets.Features({
    "instruction": datasets.Value("string"),
    "inputs": {
        "image": datasets.Image(decode=False),
        "annotation": datasets.Value("string")
    },
    "outputs": datasets.Value("string"),
    "meta": {
        "id": datasets.Value("int32"),
        'categories': {
            'topic': datasets.Value("string")
        },
        'image': {
            'source': datasets.Value("string"),
            'type': datasets.Value("string"),
            'content': datasets.Value("string")
        }
    },
})

#### Создание датасетов для каждого сплита

Теперь создаем сплиты датасета. Можно это сделать либо в одну строку:

In [43]:
shots_ds = datasets.Dataset.from_list(shots, features=features)

In [44]:
test_ds = datasets.Dataset.from_list(test, features=features)

##### Проверка

Если вы собирали датасет по кускам, то разумно будет проверить, что сборка прошла успешно - ничего не потеряно, не продублировано и так далее.

Но вы можете проверить целостность датасета даже, если и не по кусочкам собирали его. Так вы можете отловить ошибки до того, как их найдут на ревью :)

In [45]:
# проверка, что id вопросов сходятся

bools = []
for i in range(len(test)):
    bools.extend([test[i]["meta"]["id"] == test_ds[i]["meta"]["id"]])
all(bools)

True

In [46]:
# проверка, что количество вопросов до конвертации и после осталось одинаковым

len(test) == len(test_ds)

True

#### Собираем сплиты в один датасет

In [47]:
dataset = datasets.DatasetDict({"shots": shots_ds, "test": test_ds})

### Загрузка датасета на ХФ

Для загрузки на ХФ вам понадобятся:
- токен. Это строка, содержащая ключик, который позволит вам записывать в репозиторий. Для получения токена на запись в репозиторий [MERA](https://huggingface.co/MERA-evaluation) напишите Алене или Артему
- путь для записи. Это тоже строка, которая содержит путь, по которому вы выложите свой датасет. Этот путь содержит название аккаунта (MERA-evaluation) и название вашего датасета. Название датасета пишите ровно так, как оно заявлено в мете! Регистр тоже имеет значение!

In [48]:
### TOKEN
token = ""
###

# читаю токен из файла
with open("/home/jovyan/artem/mera_obs/token.txt") as f:
    token = f.read()

### UPLOAD PATH
if WITH_WATERMARK:
    dataset_path_hub = "MERA-evaluation/ruMathVQA"
else:
    dataset_path_hub = "MERA-evaluation/ruMathVQA_clear"
###

#### WARNING! 

Не забудьте флаг _private_. Он отвечает за то, что по пути _dataset_path_hub_ будет создан **приватный** репозиторий с датасетом. То есть датасет будет виден только тем, кто может зайти в аккаунт __MERA-evaluation__, а также тем, у кого есть токен на чтение/запись в этом репо.

In [49]:
dataset.push_to_hub(dataset_path_hub, private=True, token=token)

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00,  7.70ba/s]
Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

Processing Files (0 / 1)                : 100%|█████████▉| 28.5MB / 28.5MB, 47.5MB/s  



Processing Files (1 / 1)                : 100%|██████████| 28.5MB / 28.5MB, 20.3MB/s  




Processing Files (1 / 1)                : 100%|██████████| 28.5MB / 28.5MB, 12.9MB/s  
New Data Upload                         : 100%|██████████| 9.83kB / 9.83kB, 4.47kB/s  
                                        : 100%|██████████| 28.5MB / 28.5MB            
Creating parquet from Arrow format: 100%|██████████| 6/6 [00:00<00:00, 12.38ba/s]
Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

Processing Files (0 / 1)                :  36%|███▌      | 50.2MB /  139MB, 50.1MB/s  
Processing Files (0 / 1)                : 100%|█████████▉|  138MB /  139MB,  115MB/s  





Processing Files (1 / 1)                : 100%|

CommitInfo(commit_url='https://huggingface.co/datasets/MERA-evaluation/ruMathVQA/commit/021d1e24f21ffa91e366f2e105671530775f54fe', commit_message='Upload dataset', commit_description='', oid='021d1e24f21ffa91e366f2e105671530775f54fe', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/MERA-evaluation/ruMathVQA', endpoint='https://huggingface.co', repo_type='dataset', repo_id='MERA-evaluation/ruMathVQA'), pr_revision=None, pr_num=None)

### Проверка того, как датасет загрузился на ХФ

После загрузки датасета будет полезно посмотреть, как его будет видеть любой человек, который после вашей загрузки его скачает. 

Загрузите датасет целиком, используя `datasets.load_dataset(dataset_path_hub)`, а затем проверьте, что:
- все поля на месте. Если у вас в датасете у разных вопросов было разное количество вариантов ответа, то теперь их везде станет одинаковое количество. Недостающие варианты ответа у каждого вопроса теперь будут прописаны, но будут иметь значение `None`. Это нормально.
- ваша модальность корректно обработалась. Если у вас в датасете были картинки, то все они должны превратиться в байткод. Не должно остаться ни одной картинки, которая не конвертирована в байты. Если у картинки есть и байты, и путь прописан (а не `None`), то это окей. `bytes` точно должны быть заполнены, `path` может быть None.
- датасет идентичен по содержанию исходному. То есть, в исходном JSON и загруженном датасете вопрос с одинаковым `id` имеет одинаково заполненные поля (кроме тех, что заполняются `None`, как описано выше).

In [ ]:
ds = datasets.load_dataset(dataset_path_hub, token=token)

In [30]:
ds

DatasetDict({
    shots: Dataset({
        features: ['instruction', 'inputs', 'outputs', 'meta'],
        num_rows: 10
    })
    test: Dataset({
        features: ['instruction', 'inputs', 'outputs', 'meta'],
        num_rows: 832
    })
})

Пример проверки двух сплитов, что в них везде картинки конвертированы в байты

In [33]:
check = []
for card in ds["shots"]:
    image_converted_to_bytes = isinstance(card["inputs"]["image"]["bytes"], bytes)
    check.extend([image_converted_to_bytes])

all(check)

True

In [34]:
check = []
for card in ds["test"]:
    image_converted_to_bytes = isinstance(card["inputs"]["image"]["bytes"], bytes)
    check.extend([image_converted_to_bytes])

all(check)

True

Пример проверки двух сплитов, что в них тексты вопросов совпадают с оригинальными

In [35]:
check = []
for idx, card in enumerate(ds["shots"]):
    same_question = shots[idx]["inputs"]["question"] == card["inputs"]["question"]
    check.extend([same_question])

all(check)

True

In [36]:
check = []
for idx, card in enumerate(ds["test"]):
    same_question = test[idx]["inputs"]["question"] == card["inputs"]["question"]
    check.extend([same_question])

all(check)

True